In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("train.csv", index_col=[0])

In [3]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df["Weight"] = np.random.rand(len(df), 1)

In [5]:
def weighted_avg(x, wts):
    return np.average(x, weights=wts, axis=0)

def weighted_std(x, wts):
    average = np.average(x, weights=wts, axis=0)
    xiance = np.average((x - average) ** 2, weights=wts, axis=0)
    return np.sqrt(xiance)

def weighted_skew(x, wts):
    """Calculates the weighted skewness"""
    return (np.average((x - weighted_avg(x, wts))**3, weights=wts, axis=0) /
            weighted_avg(x, wts)**(1.5))

def weighted_kurtosis(x, wts):
    """Calculates the weighted skewness"""
    return (np.average((x - weighted_avg(x, wts))**4, weights=wts, axis=0) /
            weighted_avg(x, wts)**(2))

In [6]:
funcs = [weighted_avg, weighted_std, weighted_skew, weighted_kurtosis]
metrics = ["LotArea", "LotFrontage", "SalePrice"]

In [7]:
def weighted_functions(df, funcs, metrics):
    names = [func.__name__ for func in funcs]
    groups = []
    
    for func in funcs:
        gb = df.groupby(["MSZoning", "SaleCondition"]).apply(lambda x: pd.Series(func(x[metrics], x["Weight"]), metrics))
        groups.append(gb)
        
    gb = df.groupby(["MSZoning", "SaleCondition"]).apply(lambda x: pd.Series(np.median(x[metrics]), metrics))
    groups.append(gb)
    names.append("median")
    
    return pd.concat(groups, axis=1, keys=names)

In [8]:
weighted_functions(df, funcs, metrics=metrics)

weighted_avg                             weighted_std  \
                             LotArea LotFrontage      SalePrice       LotArea   
MSZoning SaleCondition                                                          
C (all)  Abnorml         8128.428483   69.924160   65834.311754    516.557832   
         Alloca          8712.000000   66.000000   55993.000000      0.000000   
         Normal          8941.890003   57.167528   87354.137520   1980.979832   
FV       Abnorml         3838.098921   36.559240  217295.808201   1008.667556   
         Normal          6512.552016         NaN  214364.060441   3122.052075   
         Partial         7246.054983         NaN  225009.446136   2449.222178   
RH       Abnorml         8198.085644   53.457020  161972.753392   2481.736885   
         Normal          7083.844495         NaN  114313.564473   2428.400126   
RL       Abnorml        11228.152200         NaN  167013.610408   4579.393186   
         AdjLand         7766.068887   56.378831   98877.376004   1277.628728   
         Alloca         11212.332570         NaN  152829.718532   6956.268156   
         Family         10722.114922         NaN  152421.697959   2170.656149   
         Normal         11736.350711         NaN  181729.248940  12741.434342   
         Partial        12325.087385   84.282590  281541.654916   6992.567890   
RM       Abnorml         6367.676734   49.239246  100081.638944   2759.563554   
         Alloca          6077.224725   53.755324  105139.953680    386.785078   
         Family          7186.771748   52.596730  138661.307381   3408.404074   
         Normal          6371.636422         NaN  124100.797258   3213.911907   
         Partial         5132.275431   48.211443  204577.967862    894.619064   

                                                 weighted_skew              \
                       LotFrontage     SalePrice       LotArea LotFrontage   
MSZoning SaleCondition                                                       
C (all)  Abnorml         18.865436  2.512275e+04  3.069761e+01   14.022568   
         Alloca           0.000000  7.275958e-12  0.000000e+00    0.000000   
         Normal          10.569750  4.138340e+04  1.989439e+04   12.168483   
FV       Abnorml         10.578750  8.307218e+04  1.084195e+03    1.346318   
         Normal                NaN  4.877603e+04 -6.040983e+03         NaN   
         Partial               NaN  5.452246e+04 -6.874100e+03         NaN   
RH       Abnorml          9.676051  3.349576e+04 -1.180171e+04   -3.226473   
         Normal                NaN  2.641108e+04  1.140975e+04         NaN   
RL       Abnorml               NaN  9.509250e+04  1.134924e+05         NaN   
         AdjLand         10.660446  2.198825e+04 -1.149365e+03   -3.811208   
         Alloca                NaN  3.639461e+04  6.765098e+05         NaN   
         Family                NaN  4.196595e+04  6.313118e+02         NaN   
         Normal                NaN  6.762666e+04  1.867496e+07         NaN   
         Partial         29.966508  1.070953e+05  1.030716e+06  129.326852   
RM       Abnorml         16.498092  2.522858e+04 -2.320774e+04   -5.928181   
         Alloca           9.541835  3.981296e+04  2.622694e+02    4.733254   
         Family          19.399800  5.028979e+04 -2.802843e+04   -2.008969   
         Normal                NaN  4.022625e+04  5.602458e+04         NaN   
         Partial         12.422050  1.252868e+05 -1.173900e+03    0.317761   

                                     weighted_kurtosis               \
                           SalePrice           LotArea  LotFrontage   
MSZoning SaleCondition                                                
C (all)  Abnorml       -3.915157e+04      1.703680e+03    71.061346   
         Alloca        -2.907166e-41      0.000000e+00     0.000000   
         Normal        -3.973230e+05      1.929900e+06   104.150542   
FV       Abnorml        3.202539e+06      1.022249e+05    13.687417   
         Normal         1.0